In [ ]:
import os
import glob
import warnings
warnings.filterwarnings("ignore")

# Colab 사용시 구동
# from google.colab import drive
# drive.mount('/content/gdrive')

import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)

# import tensorflow
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, recall_score, precision_score, f1_score

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.rc('font', family='NanumGothic')
mpl.rcParams['axes.unicode_minus'] = False
plt.style.use('ggplot')

In [ ]:
# Colab
# X_path, y_path = glob.glob('gdrive/My Drive/Gene Expression Prediction/data/train/*')

# Local
X_path, y_path = glob.glob('../data/train/*train.csv')
X_train, y_train = pd.read_csv(X_path), pd.read_csv(y_path)

X_path, y_path = glob.glob('../data/test/*test.csv')
X_test, y_test = pd.read_csv(X_path), pd.read_csv(y_path)

### H3K4me3
- DNA 패키징 단백질 히스톤 H3에 대한 후성유전적 변형
- 히스톤 H3 단백질(H3)의 4번째 라이신 잔기(K4)에서 트릴메틸화 변형(me3)을 나타냄
	- 라이신: K로 표기, HO2CCH (NH2) (CH2) 4NH2를 갖는 α- 아미노산, 필수 아미노산
	- 트릴메틸화 변형: 3개의 메틸 그룹이 히스톤이 붙는 것, me3
	- 3개의 메틸그룹이 H3K4에 붙어 H3K4me3가 히스톤 테일 프로모터 부분에 많이 있게 됨
- 유전자 발현 조절에 관여하는 마커(전사활성화(transcription activation) 마커)


### H3K4me1
- DNA 패키징 단백질 Histone H3에 대한 후성유전적 변형
- 히스톤 H3 단백질(H3)의 4번째 라이신 잔기(K4)에서 모노메틸화(me1) 변형을 나타냄
- 유전자 인핸서와 관련 있는 표시
	- 인핸서(Enhancer): 프로모터에서 멀리 떨어진 위치에서 유전자의 전사(transcription)를 조절하는 DNA 염기서열
	- [Reference](http://www.ksmcb.or.kr/file/webzine/2015_08_03.pdf)

### H3K36me3
- DNA 패키징 단백질 Histone H3에 대한 후성유전적 변형
- 히스톤 H3 단백질의 36번째 라이신 잔기(K36)에서 트릴메틸화 변형(me3)을 나타냄
- 유전자 본체와 관련 있는 표시


### H3K9me3
- DNA 패키징 단백질 Histone H3에 대한 후성유전적 변형
- 히스톤 H3 단백질의 9번째 라이신 잔기(K9)에서 트릴메틸화(me3)를 나타냄
- 헤테로크로마틴과 관련 있는 표시
	- 헤테로크로마틴(Heterocromatin): DNA가 히스톤을 빠듯하게 감고 있는 뉴클레오좀 상태
	- 유크로마틴(Euchromatin): DNA가 히스톤을 느슨하게 감고 있는 뉴클레오좀 상태
	- 히스톤 꼬리(tail)에 달라 붙는 효소와 단백질에 따라 뉴클레오좀 상태가 결정됨. 즉, 달라붙은 단백질를 표식(Marker)으로 삼아 뉴클레오좀 상태를 가늠할 수 있음

### H3K27me3
- DNA 패키징 단백질 Histone H3에 대한 후성유전적 변형
- 히스톤 H3 단백질의 27 번째 라이신 잔기에서 트리메틸 화를 나타내는 마크
- 트리메틸화는 헤테로크로마틱 영역 구성을 통해 근처 유전자의 하향조절(downregulation)과 관련

In [ ]:
train_raw = X_train.merge(y_train, how='left', on='GeneId')
train_raw.head(3)

In [ ]:
yesExpression = train_raw[train_raw['Prediction']==1].reset_index(drop=True)
noExpression = train_raw[train_raw['Prediction']==0].reset_index(drop=True)

In [ ]:
temp = train_raw.groupby(['Prediction']).median().drop('GeneId', axis=1)
temp = temp.reset_index()
temp = pd.melt(temp, id_vars='Prediction')

plt.figure(figsize=(10, 6))
sns.barplot(x='variable', y='value', hue='Prediction', data=temp)
plt.title('H3 Modification Region Index (Median)')

In [ ]:
temp = train_raw.groupby(['Prediction']).mean().drop('GeneId', axis=1)
temp = temp.reset_index()
temp = pd.melt(temp, id_vars='Prediction')

plt.figure(figsize=(10, 6))
sns.barplot(x='variable', y='value', hue='Prediction', data=temp)
plt.title('H3 Modification Region Index (Mean)')

In [ ]:
temp = train_raw.groupby(['Prediction']).std().drop('GeneId', axis=1)
temp = temp.reset_index()
temp = pd.melt(temp, id_vars='Prediction')

plt.figure(figsize=(10, 6))
sns.barplot(x='variable', y='value', hue='Prediction', data=temp)
plt.title('H3 Modification Region Index (std)')

In [ ]:
feature_list = ['H3K4me3', 'H3K4me1', 'H3K36me3', 'H3K9me3', 'H3K27me3']

def plotDist(feature=feature_list[0], figsize=(10,6), estimator='mean'):
    if estimator=='mean':
        plt.figure(figsize=(10,6))
        temp = train_raw[['GeneId', f'{feature}', 'Prediction']].groupby('GeneId').mean()
        temp_yesExp = temp[temp['Prediction']==1]
        temp_noExp = temp[temp['Prediction']==0]
        sns.distplot(temp_yesExp[f'{feature}'], norm_hist=True, label='Yes Expression')
        sns.distplot(temp_noExp[f'{feature}'], norm_hist=True, label='No Expression')
        plt.legend()
        plt.show()
    elif estimator=='median':
        plt.figure(figsize=(10,6))
        temp = train_raw[['GeneId', f'{feature}', 'Prediction']].groupby('GeneId').median()
        temp_yesExp = temp[temp['Prediction']==1]
        temp_noExp = temp[temp['Prediction']==0]
        sns.distplot(temp_yesExp[f'{feature}'], norm_hist=True, label='Yes Expression', kde=False)
        sns.distplot(temp_noExp[f'{feature}'], norm_hist=True, label='No Expression', kde=False)
        plt.legend()
        plt.show()

In [ ]:
for f in feature_list:
    plotDist(feature=f, figsize=(5,3))

H3K9me3, H3K36me3, H3K4me1: 유전자 발현 여부에 따라 확연한 분포 차이를 보임

In [ ]:
for f in feature_list:
    plotDist(feature=f, estimator='median', figsize=(5,3))

In [ ]:
train_TCN = pd.DataFrame(
    np.vstack(
        (
            train_raw
         .groupby('GeneId')
         .apply(lambda x: x.drop(['GeneId', 'Prediction'], axis=1)
        .values.reshape(500, 1)))
        .apply(lambda x: x.flatten())
        .values
        )
)
train_TCN['GeneId'] = train_raw['GeneId'].unique().tolist()
train_TCN = (
    train_TCN.merge(
        train_raw
        .groupby('GeneId')['Prediction']
        .first().to_frame('Prediction')
        .reset_index(), 
        how='left', 
        on='GeneId'
        )
    )

# Colab export data
# path = 'gdrive/My Drive/Gene Expression Prediction/data/'
# train_TCN.to_csv(os.path.join(path, 'train_TCN.csv'), index=False)

# Local export data
train_TCN.to_csv('../data/train/train_TCN.csv', index=False)

train_TCN = pd.read_csv('../data/train/train_TCN.csv').drop('GeneId', axis=1)
train_TCN.head(3)

In [ ]:
X, y = train_TCN.drop('Prediction', axis=1), train_TCN['Prediction']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=.3, random_state=42, shuffle=True)

### XGBoost

In [ ]:
model = (
    xgb.XGBClassifier(random_state=42)
    .fit(X_train.values, y_train)
         )
pred = model.predict(X_valid.values)
pred_proba = model.predict_proba(X_valid.values)[:, 1]

print(roc_auc_score(y_valid, pred_proba))
print(recall_score(y_valid, pred))
print(precision_score(y_valid, pred))
print(f1_score(y_valid, pred))

In [ ]:
kf = KFold(n_splits=4, random_state=42, shuffle=True)
auc_list = []
for train_index, test_index in kf.split(train):
    X_train, X_test = train.iloc[train_index.tolist(), :].drop('Prediction', axis=1), train.iloc[test_index.tolist(), :].drop('Prediction', axis=1)
    y_train, y_test = train['Prediction'].iloc[train_index.tolist()], train['Prediction'].iloc[test_index.tolist()]

    model = (
        xgb.XGBClassifier(random_state=42)
        .fit(X_train.values, y_train)
        )
    # AUROC
    auc_list.append(roc_auc_score(y_test, 
              model.predict_proba(X_test.values)[:, 1]))

### LightGBM

Boosting Type: GOSS

In [ ]:
model = (
    lgb.LGBMClassifier(random_state=42, boosting_type='goss')
    .fit(X_train.values, y_train)
         )

pred = model.predict(X_valid.values)
pred_proba = model.predict_proba(X_valid.values)[:, 1]

print(roc_auc_score(y_valid, pred_proba))
print(recall_score(y_valid, pred))
print(precision_score(y_valid, pred))
print(f1_score(y_valid, pred))

In [ ]:
kf = KFold(n_splits=4, random_state=42, shuffle=True)
auc_list = []
for train_index, test_index in kf.split(train):
    X_train, X_test = train.iloc[train_index.tolist(), :].drop('Prediction', axis=1), train.iloc[test_index.tolist(), :].drop('Prediction', axis=1)
    y_train, y_test = train['Prediction'].iloc[train_index.tolist()], train['Prediction'].iloc[test_index.tolist()]

    model = (
        lgb.LGBMClassifier(random_state=42, boosting_type='goss')
        .fit(X_train.values, y_train)
        )
    # AUROC
    auc_list.append(roc_auc_score(y_test, 
              model.predict_proba(X_test.values)[:, 1]))

Boosting Type: GBDT

In [ ]:
model = (
    lgb.LGBMClassifier(random_state=42, boosting_type='gbdt')
    .fit(X_train.values, y_train)
         )

pred = model.predict(X_valid.values)
pred_proba = model.predict_proba(X_valid.values)[:, 1]

print(roc_auc_score(y_valid, pred_proba))
print(recall_score(y_valid, pred))
print(precision_score(y_valid, pred))
print(f1_score(y_valid, pred))

In [ ]:
kf = KFold(n_splits=4, random_state=42, shuffle=True)
auc_list = []
for train_index, test_index in kf.split(train):
    X_train, X_test = train.iloc[train_index.tolist(), :].drop('Prediction', axis=1), train.iloc[test_index.tolist(), :].drop('Prediction', axis=1)
    y_train, y_test = train['Prediction'].iloc[train_index.tolist()], train['Prediction'].iloc[test_index.tolist()]

    model = (
        lgb.LGBMClassifier(random_state=42, boosting_type='gbdt')
        .fit(X_train.values, y_train)
        )
    # AUROC
    auc_list.append(roc_auc_score(y_test, 
              model.predict_proba(X_test.values)[:, 1]))

### SVM

In [ ]:
# 알고리즘 특성상 스케일링이 필요함
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)

In [ ]:
model = (
    SVC(random_state=42, probability=True)
    .fit(X_train, y_train)
         )

pred = model.predict(X_valid)
pred_proba = model.predict_proba(X_valid)[:, 1]

print(roc_auc_score(y_valid, pred_proba))
print(recall_score(y_valid, pred))
print(precision_score(y_valid, pred))
print(f1_score(y_valid, pred))

### Random Forest

In [ ]:
model = (
    RandomForestClassifier(random_state=42)
    .fit(X_train.values, y_train)
         )

pred = model.predict(X_valid.values)
pred_proba = model.predict_proba(X_valid.values)[:, 1]

print(roc_auc_score(y_valid, pred_proba))
print(recall_score(y_valid, pred))
print(precision_score(y_valid, pred))
print(f1_score(y_valid, pred))

In [ ]:
kf = KFold(n_splits=4, random_state=42, shuffle=True)
auc_list = []
for train_index, test_index in kf.split(train):
    X_train, X_test = train.iloc[train_index.tolist(), :].drop('Prediction', axis=1), train.iloc[test_index.tolist(), :].drop('Prediction', axis=1)
    y_train, y_test = train['Prediction'].iloc[train_index.tolist()], train['Prediction'].iloc[test_index.tolist()]

    model = (
        RandomForestClassifier(random_state=42)
        .fit(X_train.values, y_train)
        )
    # AUROC
    auc_list.append(roc_auc_score(y_test, 
              model.predict_proba(X_test.values)[:, 1]))